# Llama-Index Quickstart

In this quickstart you will create a simple Llama Index App and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/llama_index_quickstart.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [5]:
%pip install -qU "trulens_eval>=0.19.2" "llama_index>0.9.17" "html2text>=2020.1.16" qdrant_client python-dotenv ipywidgets streamlit_jupyter
# %pip install -qU trulens_eval==0.19.1 llama_index>0.9.15 html2text>=2020.1.16 qdrant_client python-dotenv

%load_ext dotenv
%dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wiki3ai 0.1.0 requires pydantic<2, but you have pydantic 2.5.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### Add API keys
For this quickstart, you will need Open AI and Huggingface keys. The OpenAI key is used for embeddings and GPT, and the Huggingface key is used for evaluation.

In [4]:
import os
from google.colab import userdata
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [10]:
import os
GOOGLE_API_KEY = os.environ["GEMINI_API_KEY"]

### Import from LlamaIndex and TruLens

In [2]:
from trulens_eval import Feedback, Tru, TruLlama
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

tru = Tru(database_redact_keys=True)

/Users/jim/Projects/wiki3ai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦑 Tru initialized with db url sqlite:///default.sqlite .
🔒 Secret keys will not be included in the database.


### Create Simple LLM Application

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [3]:
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(
    html_to_text=True
).load_data(["http://paulgraham.com/worked.html"])
documents

[Document(id_='749b861a-8694-4f91-80ec-230bedad6be9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='e63f93d1899b11fdd72d48ae487da624571439d97f87ca63fa5cc9753df3549e', text='![](https://s.turbifycdn.com/aah/paulgraham/essays-5.gif)|\n![](https://sep.turbifycdn.com/ca/Img/trans_1x1.gif)|\n[![](https://s.turbifycdn.com/aah/paulgraham/essays-6.gif)](index.html)  \n  \n| ![What I Worked On](https://s.turbifycdn.com/aah/paulgraham/what-i-worked-\non-4.gif)  \n  \nFebruary 2021  \n  \nBefore college the two main things I worked on, outside of school, were\nwriting and programming. I didn\'t write essays. I wrote what beginning writers\nwere supposed to write then, and probably still are: short stories. My stories\nwere awful. They had hardly any plot, just characters with strong feelings,\nwhich I imagined made them deep.  \n  \nThe first programs I tried writing were on the IBM 1401 that our school\ndistrict used for what

In [12]:

from llama_index import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings import GeminiEmbedding
from llama_index.llms import Gemini
from llama_index.vector_stores import QdrantVectorStore
import qdrant_client
# from llama_index.vector_stores import ChromaVectorStore
# import chromadb
from llama_index import StorageContext

# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(path="qdrant_gemini_3")

vector_store = QdrantVectorStore(client=client, collection_name="collection")

# # initialize client, setting path to save data
# db = chromadb.PersistentClient(path="./chroma_db")

# # create collection
# chroma_collection = db.get_or_create_collection("quickstart")

# # assign chroma as the vector_store to the context
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Using the embedding model to Gemini
embed_model = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)
service_context = ServiceContext.from_defaults(
    llm=Gemini(api_key=GOOGLE_API_KEY), embed_model=embed_model
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
    storage_context=storage_context,
    show_progress=True,
)

query_engine = index.as_query_engine()

Generating embeddings: 100%|██████████| 23/23 [00:05<00:00,  4.40it/s]


### Send your first request

In [13]:
response = query_engine.query("What does the author say about their education?")
print(response)

The author initially studied philosophy in college but found it boring and switched to AI. They then pursued a PhD in computer science at Harvard while also taking art classes. After completing their PhD, they applied to art schools and was accepted into the BFA program at RISD. They also received an invitation to take the entrance exam at the Accademia di Belli Arti in Florence, which they passed. However, the author ultimately decided to attend RISD.


In [14]:
response = query_engine.query("Where did the author go to school?")
print(response)

The author attended Harvard University for a PhD program in computer science, the Rhode Island School of Design (RISD) for a BFA program, and the Accademia di Belli Arti in Florence for an entrance exam.


In [15]:
response = query_engine.query("Who was the author's Harvard PhD advisor?")
print(response)

The provided context does not mention the author's Harvard PhD advisor, so I cannot answer this question from the provided context.


In [61]:
response = query_engine.query("who was Tom Cheatham to the author?")
print(response)

Tom Cheatham was the author's advisor in the PhD program in computer science at Harvard.


In [63]:
response = query_engine.query("who is Tom? why is he in this story?")
print(response)

Tom Cheatham is a professor at Harvard University. He is mentioned in the story because the narrator, who is a PhD student in computer science at Harvard, takes art classes at the university and Cheatham is his advisor. Cheatham is described as being very easy going and not saying anything about the narrator's strange class choices.


In [16]:
response = query_engine.query("what is this story about?  what are the most important things the author want the reader to learn?")
print(response)

This story is about the author's journey in pursuing his passion for artificial intelligence (AI) and his experiences in the field. The author initially had high hopes for AI, believing it could lead to the discovery of ultimate truths. However, he later realized that the AI practices at the time were limited and not capable of achieving true understanding of natural language.

The author's most important message to the reader is that prestige is a danger sign. He emphasizes the importance of focusing on the practical aspects of technology and not being swayed by prestige or high-end solutions. He believes that the "low end" or entry-level options often have the potential to disrupt and eventually overtake the high end.


## Initialize Feedback Function(s)

In [17]:
import numpy as np

# Initialize provider class
openai = OpenAI()

grounded = Groundedness(groundedness_provider=openai)

# Define a groundedness feedback function
f_groundedness = Feedback(grounded.groundedness_measure_with_cot_reasons).on(
    TruLlama.select_source_nodes().node.text.collect()
    ).on_output(
    ).aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


## Instrument app for logging with TruLens

In [19]:
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

In [24]:
# or as context manager
with tru_query_engine_recorder as recording:
    response = query_engine.query("Why did the author drop AI?")
    print(response)

The author dropped AI because they realized that the way AI was practiced at the time was a hoax. They believed that the whole way of doing AI, with explicit data structures representing concepts, was not going to work and would never lead to the creation of truly intelligent machines like Mike from the novel _The Moon is a Harsh Mistress_.


## Explore in a Dashboard

In [27]:
# tru.run_dashboard() # open a local streamlit app to explore

tru.run_dashboard_in_jupyter() # open a streamlit app in the notebook

Running dashboard inside a notebook is an experimental feature and may not work well.



        <style>
            [data-testid="stSidebarNav"] {
                background-image: url('');
                background-repeat: no-repeat;
                background-size: 300px auto;
                padding-top: 50px;
                background-position: 20px 20px;
            }
            [data-testid="stSidebarNav"]::before {
                margin-left: 20px;
                margin-top: 20px;
                font-size: 30px;
                position: relative;
                top: 100px;
            }
            [data-testid="stSidebarNav"]::after {
                margin-left: 20px;
                color: #aaaaaa;
                content: "trulens_eval 0.19.2";
                font-size: 10pt;
            }

            /* For list items in st.dataframe */
            #portal .clip-region .boe-bubble {
                height: auto;
                border-radius: 4px;
                padding: 8px;
            }
        </style>
        

# App Leaderboard

Average feedback values displayed in the range from 0 (worst) to 1 (best).

---

...

2023-12-20 03:54:02.354 
  command:

    streamlit run /Users/jim/Projects/wiki3ai/.venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]



    <style> [data-testid="stMetricDelta"] svg { display: none; } </style>
    


    <style> [data-testid="stMetricDelta"] svg { display: none; } </style>
    


    <style> [data-testid="stMetricDelta"] svg { display: none; } </style>
    

---

In [ ]:
tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

Note: Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.

## Or view results directly in your notebook

In [26]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,qs_relevance,groundedness_measure_with_cot_reasons,relevance_calls,qs_relevance_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1cb4a6448e269734eb06c48ca37dbd83,"""What did the author do growing up?""","""The provided context does not contain informa...",-,"{""record_id"": ""record_hash_1cb4a6448e269734eb0...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-20T03:48:31.211610"", ""...",2023-12-20T03:48:34.677256,1.0,0.2,0.0,[{'args': {'prompt': 'What did the author do g...,[{'args': {'question': 'What did the author do...,[{'args': {'source': ['But the most important ...,3,0,0.0
1,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_d8e36d766bb851ed3c1c4a53c8c252aa,"""Why did the author drop AI?""","""The author dropped AI because they realized t...",-,"{""record_id"": ""record_hash_d8e36d766bb851ed3c1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-20T03:52:55.666568"", ""...",2023-12-20T03:53:02.229297,0.8,0.2,1.0,[{'args': {'prompt': 'Why did the author drop ...,[{'args': {'question': 'Why did the author dro...,[{'args': {'source': ['Though I liked programm...,6,0,0.0
